Impact Ann

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import re

In [ ]:
tf.__version__

Load data

In [ ]:
!curl -L -o DeviceAbuseDataImpact.zip https://github.com/arnabalumnus/Zebra/raw/code_cleanup/python/DeviceAbuseDataImpact.zip

In [ ]:
import zipfile
with zipfile.ZipFile('DeviceAbuseDataImpact.zip', 'r') as zip_ref:
    zip_ref.extractall('./')

In [ ]:
# Create a folder with name `DeviceAbuseDataInpact`
# Copy all data .csv file into this folder before running

def preProcess(rowCount):
    trainingDataFolder = "DeviceAbuseDataImpact/"

    dirPath = os.path.join(os.getcwd(), trainingDataFolder)
    files = [f for f in os.listdir(dirPath) if os.path.isfile(os.path.join(dirPath, f))]

    fileList = []
    finalDF = pd.DataFrame()
    
    for file in files:
        # Importing the dataset
        orgData = pd.read_csv(dirPath + file)
        data = pd.DataFrame(orgData.iloc[:, 1:4])

        print('Current File :', file, '# Rowcount :', data.shape[0])       
        if data.shape[0] > rowCount:
            fileList.append(file)
            start = 0
            end = rowCount
           
            count = 0
            for i in range(start, end):
                temp = pd.DataFrame(data.iloc[i:i+1, 0:3])
                temp.columns = ['X'+str(i), 'Y'+str(i), 'Z'+str(i)]
                temp = temp.reset_index(drop=True)
               
                if count == 0:
                    tempDF = temp
                    count = 1
                else:
                    tempDF = pd.concat([tempDF.reset_index(drop=True), temp], axis=1)
                   
            finalDF = finalDF.append(tempDF)
        elif (data.shape[0] >= (rowCount / 2)):
            fileList.append(file)
            diff = rowCount - data.shape[0]
            start = 0
            end = rowCount
            count = 0
            countDown = rowCount - data.shape[0]

            for i in range(start, end):
                if (countDown > 0):
                    temp = pd.DataFrame(data.iloc[0:1, 0:3])
                    countDown -= 1
                else:
                    temp = pd.DataFrame(data.iloc[(i - diff):(i + 1 - diff), 0:3])

                temp.columns = ['X'+str(i), 'Y'+str(i), 'Z'+str(i)]
                temp = temp.reset_index(drop=True)
               
                if count == 0:
                    tempDF = temp
                    count = 1
                else:
                    tempDF = pd.concat([tempDF.reset_index(drop=True), temp], axis=1)
                   
            finalDF = finalDF.append(tempDF)
        else:
            print("No. of rows is less than", (rowCount / 2))
           
    fileList = [i.split('_', 1)[0] for i in fileList]
    fileList = [re.sub("\d+", "", i) for i in fileList]
    actualOutput = pd.DataFrame([fileName[:4] for fileName in fileList])
    actualOutput.columns = ["actualOutput"]
    
    finalDF = finalDF.reset_index(drop=True)
    finalDF = pd.concat([finalDF, actualOutput], axis=1)
    
    return finalDF

In [ ]:
finalDF = preProcess(100)

In [ ]:
print(finalDF)

In [ ]:
X=finalDF.iloc[:, :-1].values

In [ ]:
print(X)

In [ ]:
y=finalDF.iloc[:, -1].values

In [ ]:
print(y)

In [ ]:
type(y)

In [ ]:
myset = set(y)
print(myset)

In [ ]:
print(y)

In [ ]:
y[y=='DSXX'] = 0
y[y=='FSXX'] = 1
y[y=='WSXX'] = 2

In [ ]:
print(y)

In [ ]:
# convert integers to dummy variables (i.e. one hot encoded)
from keras.utils import np_utils

#convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(y)

In [ ]:
print(dummy_y)

Splitting the dataset into the Training set and Test set


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, dummy_y, test_size = 0.2, random_state = 0)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
print(X_train)

In [ ]:
print(y_train)

In [ ]:
ann = tf.keras.models.Sequential()

In [ ]:
ann.add(tf.keras.layers.Dense(units=600, activation='relu'))

In [ ]:
ann.add(tf.keras.layers.Dense(units=3, activation='softmax'))

In [ ]:
ann.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
ann.fit(X_train, y_train, batch_size = 5, epochs = 100)

In [ ]:
print(X_test)

In [ ]:
y_pred = ann.predict(X_test)


In [ ]:
print(y_pred)

In [ ]:
index = tf.argmax(y_pred, axis=1)

In [ ]:
outcome = index.numpy()


In [ ]:
print(outcome)

Read from test data set 

In [ ]:
testDataFolder = "DeviceAbuseDataFallTest"
testDataFile = "FF03YPU_2017_08_17_15_40_36  - prefall.csv"
dirPath = os.path.join(os.getcwd(), testDataFolder)

In [ ]:
print(dirPath)

In [ ]:
testDataFileWithPath = os.path.join(dirPath, testDataFile)

In [ ]:
print(testDataFileWithPath)

In [ ]:
!curl -L -o test https://raw.githubusercontent.com/arnabalumnus/Zebra/filter_data_chunk/python/DeviceAbuseDataImpactTest/DSXXXX_2017_11_06_09_19_04%20%20-%20preimpact.csv

In [ ]:
testDF = pd.DataFrame()
testData = pd.read_csv('test')

In [ ]:
print(testData)

In [ ]:
testData1 = pd.DataFrame(testData.iloc[:, 1:4])

In [ ]:
print(testData1)

In [ ]:
testData2 = pd.DataFrame()
count = 0
for i in range(0, 100):
  temp = pd.DataFrame(testData1.iloc[i:i+1, 0:3])
  temp.columns = ['X'+str(i), 'Y'+str(i), 'Z'+str(i)]
  temp = temp.reset_index(drop=True)
               
  if count == 0:
    tempDF = temp
    count = 1
  else:
    tempDF = pd.concat([tempDF.reset_index(drop=True), temp], axis=1)
            
               

In [ ]:
print(tempDF)

In [ ]:
prediction_result = ann.predict(tempDF)

In [ ]:
print(prediction_result)

In [ ]:
prediction_Value = tf.argmax(prediction_result, axis=1)

In [ ]:
print(prediction_Value)

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(ann)
tflite_model = converter.convert()

In [ ]:
with open('neural_network_impact_data_model.tflite', 'wb') as f:
  f.write(tflite_model)